## imports

In [1]:
import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import single, complete, average, ward, dendrogram,fcluster
from datetime import date,datetime,timedelta
import matplotlib.pyplot as plt

# Data preperation 

In [2]:
#read in whole data excluding Linz csv file :
df = pd.read_csv("/Users/tamo/Documents/Studium/WS 20-21/Master thesis/Data/Intelliekon/consumption_loadcourse_excl_Linz.csv")
#read in only Ulm data csv file:
#df = pd.read_csv("/Users/tamo/Documents/Studium/WS 20-21/Master thesis/Data/Intelliekon/consumption_loadcurse_Ulm.csv")

/Users/tamo/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df["datetime"] = pd.to_datetime(df["datetime"])

df["day_of_year"] = df["datetime"].apply(lambda x: x.timetuple().tm_yday)

In [4]:
#filtering out all rows with zero values for Electricity_Demand
#df = df[df["Electricity_Demand"] != 0]

In [5]:
#normalisation:
df['max_energy'] = df.groupby(['Fall_ge1'])['Electricity_Demand'].transform(max)
df['norm_energy'] = df['Electricity_Demand'] / df['max_energy']

In [6]:
#Meteorological seasons:
#winter
sdate_wi = date(2009, 12, 1)   # start date
edate_wi = date(2010, 2, 28)   # end date
#spring
sdate_sp = date(2010, 3, 1)   # start date
edate_sp = date(2010, 5, 31)   # end date
#summer
sdate_su = date(2010, 6, 1)   # start date
edate_su = date(2010, 8, 31)   # end date
#Fall
sdate_fa = date(2009, 9, 1)   # start date
edate_fa = date(2009, 11, 30)   # end date

In [7]:
#split df by season:
df_fa = df[(df["datetime"] > str(sdate_fa)) & (df["datetime"] <= str(edate_fa))]
df_wi = df[(df["datetime"] > str(sdate_wi)) & (df["datetime"] <= str(edate_wi))]
df_sp = df[(df["datetime"] > str(sdate_sp)) & (df["datetime"] <= str(edate_sp))]
df_su = df[(df["datetime"] > str(sdate_su)) & (df["datetime"] <= str(edate_su))]

In [8]:
#split dataframes into weekday and weekend days
df_fa_week=df_fa[df_fa["Weekday"]==1]
df_fa_wknd=df_fa[df_fa["Weekday"]==2]
df_wi_week=df_wi[df_wi["Weekday"]==1]
df_wi_wknd=df_wi[df_wi["Weekday"]==2]
df_sp_week=df_sp[df_sp["Weekday"]==1]
df_sp_wknd=df_sp[df_sp["Weekday"]==2]
df_su_week=df_su[df_su["Weekday"]==1]
df_su_wknd=df_su[df_su["Weekday"]==2]

In [9]:
df_fa_week.shape

(659064, 16)

In [10]:
df_fa_wknd.shape

(270648, 16)

In [11]:
# #create numpy array with mean for every hour of each ID as columns
# red_df=df_fa_week[["Fall_ge1","day_of_year","Hour_of_Day","norm_energy"]]
# piv_df = pd.pivot_table(red_df,index = ["Hour_of_Day","day_of_year"],columns = "Fall_ge1",values="norm_energy")
# grp_df = piv_df.groupby("Hour_of_Day").mean()
# cluster_df = grp_df
# cluster_df

In [12]:
# import scipy.cluster.hierarchy as sc

# # Plot dendrogram
# plt.figure(figsize=(20, 7))  
# plt.title("Dendrograms")  

# # Create dendrogram
# sc.dendrogram(sc.linkage(cluster_df, method='ward'))

# plt.title('Dendrogram')
# plt.xlabel('Sample index')
# plt.ylabel('Euclidean distance')

# Function for hierarchical clustering 

In [13]:
#funktion for clustering, select method and plot of dendrogram
def hierarchical_clustering(dist_mat, method='ward'):
    if method == 'complete':
        Z = complete(distance_matrix)
    if method == 'single':
        Z = single(distance_matrix)
    if method == 'average':
        Z = average(distance_matrix)
    if method == 'ward':
        Z = ward(distance_matrix)
    
    fig = plt.figure(figsize=(16, 8))
    dn = dendrogram(Z)
    plt.title(f"Dendrogram for {method}-linkage with correlation distance")
    plt.show()
    
    return Z

# Fall

## Fall weekdays 

In [14]:
#create numpy array with mean for every hour of each ID as columns
red_df=df_fa_week[["Fall_ge1","day_of_year","Hour_of_Day","Electricity_Demand"]]
piv_df = pd.pivot_table(red_df,index = ["Hour_of_Day","day_of_year"],columns = "Fall_ge1",values="Electricity_Demand")
grp_df = piv_df.groupby("Hour_of_Day").mean()
cluster_df = grp_df
cluster_df

Fall_ge1,1,2,3,4,5,6,9,10,12,13,...,1360,1361,1362,1363,1364,1365,1366,1367,1370,1371
Hour_of_Day,,,,,,,,,,,,,,,,,,,,,
1,201.037037,331.24,513.886792,144.048387,374.188679,87.344828,213.160714,683.907407,175.709677,1303.207547,...,126.142857,124.800000,181.363636,144.24,61.173913,114.466667,184.88,80.142857,92.250000,25.325
2,200.703704,322.20,502.283019,145.258065,305.037736,88.672414,184.142857,481.833333,109.741935,1292.490566,...,119.964286,136.500000,178.606061,142.72,65.043478,113.066667,146.00,79.357143,50.642857,25.175
3,187.962963,322.00,492.584906,144.177419,260.811321,88.879310,155.482143,475.759259,90.951613,1302.188679,...,125.607143,126.566667,187.939394,141.72,59.956522,123.800000,137.68,69.678571,49.642857,25.950
4,188.666667,328.96,497.735849,162.709677,249.716981,93.793103,153.678571,467.944444,84.758065,1305.056604,...,143.071429,127.433333,171.151515,125.28,66.043478,144.933333,132.52,74.214286,51.785714,24.450
5,214.203704,330.06,491.490566,312.806452,254.830189,100.120690,205.464286,532.351852,86.532258,1474.698113,...,152.892857,120.966667,176.848485,170.96,181.304348,148.933333,161.56,81.714286,47.964286,24.475
6,311.462963,333.28,501.811321,154.096774,316.433962,252.706897,255.053571,594.870370,86.790323,1535.849057,...,312.964286,458.300000,189.606061,659.72,433.608696,466.000000,461.08,221.214286,142.392857,24.600
7,342.574074,340.84,510.264151,187.290323,369.320755,384.603448,296.428571,523.425926,76.854839,2050.018868,...,349.428571,400.400000,289.333333,647.72,197.130435,366.533333,607.04,173.821429,83.428571,108.825
8,478.296296,476.50,529.018868,212.532258,286.452830,496.241379,351.339286,626.555556,164.483871,2245.018868,...,356.035714,372.133333,243.515152,418.68,188.000000,304.533333,385.00,211.321429,106.642857,243.775
9,496.185185,594.16,585.622642,253.403226,415.754717,790.206897,469.678571,933.074074,265.419355,2400.622642,...,328.214286,411.466667,243.000000,346.36,201.869565,357.933333,755.68,202.250000,179.035714,112.950


In [15]:
#create distance matrix for the cluster algorithm
distance_matrix = cluster_df.corr()
#len(df)
#distance_matrix
#len(df) - distance_matrix.count()
distance_matrix.isnull().sum()

Fall_ge1
1       1
2       1
3       1
4       1
5       1
       ..
1365    1
1366    1
1367    1
1370    1
1371    1
Length: 547, dtype: int64

In [16]:
distance_matrix

Fall_ge1,1,2,3,4,5,6,9,10,12,13,...,1360,1361,1362,1363,1364,1365,1366,1367,1370,1371
Fall_ge1,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.623591,0.558822,0.755630,0.538037,0.603599,0.651842,0.448308,0.588399,0.722510,...,0.720164,0.769212,0.656749,0.673448,0.621046,0.598057,0.816209,0.761774,0.666548,0.507765
2,0.623591,1.000000,0.280919,0.604230,0.439502,0.348820,0.497788,0.339347,0.605837,0.694369,...,0.252469,0.460450,0.153800,0.566371,0.620272,0.286034,0.626091,0.367269,0.624205,0.087766
3,0.558822,0.280919,1.000000,0.693370,0.694983,0.211908,0.859353,0.686641,0.362730,0.087807,...,0.579235,0.777269,0.808519,0.468550,0.510131,0.464210,0.595346,0.402996,0.721186,0.611641
4,0.755630,0.604230,0.693370,1.000000,0.729688,0.376974,0.880532,0.466752,0.441777,0.507751,...,0.586885,0.824691,0.743220,0.712665,0.733434,0.552090,0.767648,0.650375,0.851301,0.471950
5,0.538037,0.439502,0.694983,0.729688,1.000000,0.266207,0.777177,0.405903,0.134909,0.294277,...,0.435947,0.710699,0.619333,0.619061,0.521965,0.325330,0.730140,0.621678,0.793848,0.251587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1365,0.598057,0.286034,0.464210,0.552090,0.325330,0.302350,0.621631,0.214767,0.174023,0.371436,...,0.819956,0.742110,0.641339,0.780338,0.765087,1.000000,0.730834,0.614910,0.575321,0.694088
1366,0.816209,0.626091,0.595346,0.767648,0.730140,0.584379,0.755060,0.369778,0.301430,0.658440,...,0.765101,0.819851,0.666314,0.872739,0.802506,0.730834,1.000000,0.777208,0.755693,0.494424
1367,0.761774,0.367269,0.402996,0.650375,0.621678,0.399424,0.580536,0.027912,0.186892,0.475001,...,0.697643,0.809401,0.665090,0.774137,0.649613,0.614910,0.777208,1.000000,0.543924,0.431117


In [17]:
linkage_matrix = hierarchical_clustering(distance_matrix)

ValueError: The condensed distance matrix must contain only finite values.

## Fall weekend days 

In [ ]:
#create numpy array with mean for every hour of each ID as columns
red_df=df_fa_wknd[["Fall_ge1","day_of_year","Hour_of_Day","Electricity_Demand"]]
piv_df = pd.pivot_table(red_df,index = ["Hour_of_Day","day_of_year"],columns = "Fall_ge1",values="Electricity_Demand")
piv_df.reset_index(inplace = True)
grp_df = piv_df.groupby("Hour_of_Day").mean()
del grp_df["day_of_year"]
cluster_df = grp_df
cluster_df

In [ ]:
# # from itertools import combinations

# # [(i, j) for i,j in combinations(cluster_df, 2) if cluster_df[i].equals(df[j])]

# def getDuplicateColumns(df):
#     '''
#     Get a list of duplicate columns.
#     It will iterate over all the columns in dataframe and find the columns whose contents are duplicate.
#     :param df: Dataframe object
#     :return: List of columns whose contents are duplicates.
#     '''
#     duplicateColumnNames = set()
#     # Iterate over all the columns in dataframe
#     for x in range(df.shape[1]):
#         # Select column at xth index.
#         col = df.iloc[:, x]
#         # Iterate over all the columns in DataFrame from (x+1)th index till end
#         for y in range(x + 1, df.shape[1]):
#             # Select column at yth index.
#             otherCol = df.iloc[:, y]
#             # Check if two columns at x 7 y index are equal
#             if col.equals(otherCol):
#                 duplicateColumnNames.add(df.columns.values[y])
#     return list(duplicateColumnNames)

# # Get list of duplicate columns
# duplicateColumnNames = getDuplicateColumns(cluster_df)
# print('Duplicate Columns are as follows')
# for col in duplicateColumnNames:
#     print('Column name : ', col)

In [ ]:
#create distance matrix for the cluster algorithm
distance_matrix = cluster_df.corr()
#len(df)
#distance_matrix
#len(df) - distance_matrix.count()
distance_matrix.isnull().sum()

In [ ]:
distance_matrix

In [ ]:
is_NaN = distance_matrix.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = distance_matrix[row_has_NaN]
print(rows_with_NaN)

In [ ]:
linkage_matrix = hierarchical_clustering(distance_matrix)